In [57]:
from bs4 import BeautifulSoup as bs 
import os
import requests
import re 
import datetime
url = 'https://www.ptt.cc/bbs/Beauty/index.html'
my_cookies = { 'cookie':' _ga=GA1.2.243592888.1551752918; __cfduid=de3e03e1c1c1d9d267c43f0da9167e12a1583417355; _gid=GA1.2.1765845257.1583765682; over18=1' }
request = requests.get( url , cookies = my_cookies )
html = bs( request.text )
#print(html.prettify())


In [58]:
def get_articles(  html, pages ):
    pre_url = html.find_all( 'a', class_="btn wide" )[1]['href'] 
    pre_url = 'https://www.ptt.cc/'+pre_url
    articles = html.find_all('div',"r-ent")
    for doc in articles:
        push_string = doc.find( 'div',"nrec" ).text
        push_count = 0
        if push_string:
            try:
                push_count = int( push_string )
            except ValueError:
                if push_string == "爆":
                    push_count = 99
                else:
                    push_count = -10
        if push_count > 10:
            if doc.find( 'a' )==None:
                break
            name = doc.find( 'a' ).text
            #print(name)
            doc_url = doc.find( 'a' )['href']
            if name[1] == '正':
                pages.append( { 'name':name, 'url':doc_url } )
    return pages, pre_url
    

In [59]:
def download_pic( pages ):
    for page in pages:
        delete_chars = '\/:*?"<>|.'
        for c in delete_chars:
            page['name'] = page['name'].replace(c,'')
        os.makedirs( './pic/'+page['name'].strip(), exist_ok = True )
        request = requests.get( 'https://www.ptt.cc/'+page['url'], cookies = my_cookies )
        html = bs( request.text )
        pics = html.find_all( 'a', {'href':re.compile('https://(imgur|i\.imgur)\.com/.......\.jpg')} )        
        for pic in pics:
            pic_name = pic.get_text().split('/')[-1]            
            with open( './pic/'+page['name'].strip()+'/'+pic_name, 'wb' ) as f:
                r = requests.get( pic.get_text() )
                f.write( r.content )
        

In [60]:
if __name__ == '__main__':
    os.makedirs('./pic', exist_ok = True)
    pages = []
    while len(pages) < 10:
        pages, pre_url = get_articles( html, pages ) 
        request = requests.get( pre_url, cookies = my_cookies )
        html = bs( request.text )
    download_pic( pages )
        
    
    